### Note:

To run in Colab, you will need to add the datasets/ folder to your Google Drive as /data/ (in your Google Drive root directory).

In [1]:
%matplotlib inline
!pip install transformers
import math
import nltk
import numpy as np
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import unittest

from collections import Counter
from datetime import datetime
from nltk.corpus import stopwords
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup

     |████████████████████████████████| 2.1MB 18.7MB/s 
     |████████████████████████████████| 870kB 51.1MB/s 
     |████████████████████████████████| 3.3MB 52.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=3996f53955b06901e8ea5e1eb9b1e1d16ed20df50dac50c194843bf2161caf2a
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


## Utils

In [2]:
def set_seed(seed, device='cuda'):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if device == 'cuda':
    torch.cuda.manual_seed_all(seed)

class TwoWayDict(dict):
    # From https://stackoverflow.com/questions/1456373/two-way-reverse-map
    def __setitem__(self, key, value):
        # Remove any previous connections with these values
        if key in self:
            del self[key]
        if value in self:
            del self[value]
        dict.__setitem__(self, key, value)
        dict.__setitem__(self, value, key)

    def __delitem__(self, key):
        dict.__delitem__(self, self[key])
        dict.__delitem__(self, key)

    def __len__(self):
        """Returns the number of connections"""
        return dict.__len__(self) // 2

### Reuters8 Dataset

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
class Reuters8Dataset(Dataset):
    '''Initializes a PyTorch Dataset with the appropriate Reuters8 data.
    
    Args:
        filename (str): The name of the file containing Ruters8 label/sentence pairs as TSV. This should be 
            one of 'training', 'test', 'validation'. The files must be located at <cwd>/datasets/Reuters8/
        label_filepath (:obj:`str`, optional): The label file containing the label ordering, separated by 
            newlines. By default, this looks to <cwd>/datasets/Reuters8/labels.txt. 
        one_hot (:obj:`bool`, optional): Set to True by default. Determines if labels are one-hot encoded 
            vectors (True) or topic strings (False)
        f_min (:obj:`int`, optional): Set to 7 by default. We remove all words with frequency < f_min.
    '''
    
    @staticmethod
    def _read_reuters_tsv(path):
        pairs = []
        with open(path) as f:
            f.readline()
            for line in f:
                label, sent = line.split('\t')
                pairs.append([sent, label])
                
        return pairs
    
    def _clean_examples(self, examples):
        # lowercases, removes NLTK stopwords, and removes words appearing <f_min times            
        cleaned = []
        stops = stopwords.words('english')
        for ex in examples:
            sent, label = ex
            new_sent = ' '.join([x.lower() for x in sent.split(' ') if x.lower() not in stops])
            cleaned.append([new_sent, label])
            
        counts = Counter([split for x in [y[0] for y in cleaned] for split in x.split(' ')])
        final = []
        for ex in cleaned:
            sent, label = ex
            new_sent = ' '.join([x for x in sent.split(' ') if counts[x] >= self.f_min])
            final.append([new_sent, label])
            
        return final
        
    
    def __init__(self, filename, label_filepath=None, one_hot=True, f_min=7):
        self.f_min = f_min
        if label_filepath is None:
            label_filepath = os.path.join('/content/drive/My Drive/data/Reuters8/', 'labels.txt')
            
        with open(label_filepath) as f:
            self.labels = [x.strip() for x in f.read().strip().split('\n')]
    
        self.label_mapping = TwoWayDict()
        for i, label in enumerate(self.labels):
            self.label_mapping[label] = i
            
        tsv_path = os.path.join('/content/drive/My Drive/data/Reuters8/', filename)
        self.examples = self._clean_examples(Reuters8Dataset._read_reuters_tsv(tsv_path))
        
        if one_hot:
            for i, ex in enumerate(self.examples):
                _, label = ex
                lbl_one_hot = torch.full((len(self.labels),), 0)
                lbl_one_hot[self.label_mapping[label]] = 1
                self.examples[i][1] =  lbl_one_hot
                
    @property
    def num_labels(self):
        return len(self.labels)

    def __getitem__(self, index):
        return self.examples[index]
    
    def __len__(self):
        return len(self.examples)
                
        
r8_train = Reuters8Dataset('training.tsv')
r8_val = Reuters8Dataset('validation.tsv')
r8_test = Reuters8Dataset('test.tsv')

if True: # Set to false to not show example
    ex = 99
    print('Sample sent/label pair — Ex #{}:'.format(ex))
    print('\t- sent:',r8_train[ex][0].strip()) 
    print('\t- one-hot:',r8_train[ex][1]) 
    # convert one-hot label to int & string
    lbl_idx = torch.where(r8_train[ex][1] > 0)[0].item()
    print('\t- int:',lbl_idx) 
    print('\t- str:',r8_train.label_mapping[lbl_idx])

Sample sent/label pair — Ex #99:
	- sent: stone inc sets quarterly qtly div cts vs cts prior pay may record april one reuter
	- one-hot: tensor([0, 0, 1, 0, 0, 0, 0, 0])
	- int: 2
	- str: earn


In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# We need to provide a custom collate function to DataLoader because we're handling sentences of varying lengths. 
def collate_fn(batch):
  # https://huggingface.co/transformers/preprocessing.html
  sents, labels = zip(*batch)
  labels = torch.stack([x.long() for x in labels])
  encoded = tokenizer(sents, padding=True, max_length=512, truncation=True, return_tensors='pt')
  # print([len(x) for x in encoded['input_ids']])
  # print(any([len(x) > 512 for x in encoded['input_ids']]))
  return encoded['input_ids'], encoded['attention_mask'], labels

set_seed(42)
dataloader_val = DataLoader(r8_val, batch_size=8, shuffle=False, num_workers=2, collate_fn=collate_fn)

if True: # Set to false to not show example
    for i, (input_ids, attention_mask, labels) in enumerate(dataloader_val):
      if i == 2:
        print(input_ids[3])
        print(tokenizer.convert_ids_to_tokens(input_ids[3]))
        print(tokenizer.decode(input_ids[3]))
        print(attention_mask[3])
        print(labels[3])


tensor([  101,  3001,  4297,  2095,  3279,  6728,  2121, 14021,  2099,  3279,
         2274, 14931,  2015,  5443,  3279, 21469,  2869,  6728,  2121,  5658,
         3279,  5443,  3279,  4341, 19875,  2078,  5443, 19875,  2078,  3602,
         5658, 23329,  2015,  6409,  8944,  3136, 21469,  2869,  5443, 21469,
         2869,  2128, 19901,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,

In [7]:
def get_init_transformer(transformer):
  """
  Initialization scheme used for transformers:
  https://huggingface.co/transformers/_modules/transformers/modeling_bert.html
  """
  def init_transformer(module):
    if isinstance(module, (nn.Linear, nn.Embedding)):
        module.weight.data.normal_(mean=0.0, std=transformer.config.initializer_range)
    elif isinstance(module, nn.LayerNorm):
        module.bias.data.zero_()
        module.weight.data.fill_(1.0)
    if isinstance(module, nn.Linear) and module.bias is not None:
        module.bias.data.zero_()

  return init_transformer


class BertClassifier(nn.Module):

  def __init__(self, num_labels, drop=0.1):
    super().__init__()
    self.encoder = BertModel.from_pretrained('bert-base-uncased')
    self.score = nn.Sequential(nn.Dropout(drop), 
                               nn.Linear(self.encoder.config.hidden_size, num_labels))
    self.score.apply(get_init_transformer(self.encoder))  # Important to initialize any additional weights the same way as pretrained encoder. 
    self.loss = nn.CrossEntropyLoss(reduction='mean')

  def forward(self, input_ids, attention_mask, labels):
    hiddens_last = self.encoder(input_ids, attention_mask=attention_mask)[0]  # (batch_size, length, dim), these are last layer embeddings
    embs = hiddens_last[:,0,:]  # [CLS] token embeddings
    logits = self.score(embs)
    # Runs cross-entropy softmax loss on labels
    loss_total = self.loss(logits, torch.max(labels, 1)[1])
    return logits, loss_total

In [8]:
def configure_optimization(model, num_train_steps, num_warmup_steps, lr, weight_decay=0.01):  
  # Copied from: https://huggingface.co/transformers/training.html
  no_decay = ['bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [{'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 
                                   'weight_decay': weight_decay},
                                  {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                                   'weight_decay': 0.}]
  optimizer = AdamW(optimizer_grouped_parameters, lr=lr)  
  scheduler = get_linear_schedule_with_warmup(optimizer, num_training_steps=num_train_steps, num_warmup_steps=num_warmup_steps) 
  return optimizer, scheduler

In [9]:
def get_acc_val(model, device):
  num_correct_val = 0
  model.eval()  
  with torch.no_grad(): 
    for input_ids, attention_mask, labels in dataloader_val:
      input_ids = input_ids.to(device)
      attention_mask = attention_mask.to(device)
      labels = labels.to(device)
      logits, _ = model(input_ids, attention_mask, labels) 
      preds = torch.max(logits, 1)[1] # get label from max score per example
      num_correct_val += (preds == torch.max(labels, 1)[1]).sum()
  acc_val = num_correct_val / len(dataloader_val.dataset) * 100.
  return acc_val

In [ ]:
def train(model, batch_size=32, num_warmup_steps=10, lr=0.00005, num_epochs=3, clip=1., verbose=True, device='cuda'):
  model = model.to(device)  # Move the model to device.  
  dataloader_train = DataLoader(r8_train, batch_size=batch_size, shuffle=True, num_workers=2, collate_fn=collate_fn) 
  num_train_steps = len(r8_train) // batch_size * num_epochs
  optimizer, scheduler = configure_optimization(model, num_train_steps, num_warmup_steps, lr)

  loss_avg = float('inf')
  acc_train = 0.
  best_acc_val = 0.
  for epoch in range(num_epochs):
    model.train()  # This turns on the training mode (e.g., enable dropout).
    loss_total = 0.
    num_correct_train = 0
    for batch_ind, (input_ids, attention_mask, labels) in enumerate(dataloader_train):
      if (batch_ind + 1) % 200 == 0: 
        print(batch_ind + 1, '/', len(dataloader_train), 'batches done')
      input_ids = input_ids.to(device).long()
      attention_mask = attention_mask.to(device)
      labels = labels.to(device)      
      logits, loss_batch_total = model(input_ids, attention_mask, labels) 
      preds = torch.max(logits, 1)[1]  # get label from max score per example
      num_correct_train += (preds == torch.max(labels, 1)[1]).sum()
      loss_total += loss_batch_total.item()            
      
      loss_batch_avg = loss_batch_total / input_ids.size(0)  
      loss_batch_avg.backward()  

      if clip > 0.:  # Optional gradient clipping
        nn.utils.clip_grad_norm_(model.parameters(), clip)

      optimizer.step()  # optimizer updates model weights based on stored gradients
      scheduler.step()  # Update lr. 
      optimizer.zero_grad()  # Reset gradient slots to zero

    # Useful training information
    loss_avg = loss_total / len(dataloader_train.dataset)
    acc_train = num_correct_train / len(dataloader_train.dataset) * 100.

    # Check validation performance at the end of every epoch. 
    acc_val = get_acc_val(model, device)

    if verbose:
      print('Epoch {:3d} | avg loss {:8.4f} | train acc {:2.2f} | val acc {:2.2f}'.format(epoch + 1, loss_avg, acc_train, acc_val))

    if acc_val > best_acc_val: 
      best_acc_val = acc_val
  
  if verbose: 
    print('Final avg loss {:8.4f} | final train acc {:2.2f} | best val acc {:2.2f}'.format(loss_avg, acc_train, best_acc_val))

  return best_acc_val

## Below Runs BERT on Reuters8

In [ ]:
!pip install ipython-autotime  # Useful library for tracking runtime on Python Notebooks
 
%load_ext autotime

time: 288 µs (started: 2021-04-15 13:35:28 +00:00)


In [ ]:
if True: # Set True to run. 
  torch.cuda.empty_cache()
  set_seed(42)
  model = BertClassifier(r8_train.num_labels)
  best_acc_val = train(model, batch_size=8)


200 / 624 batches done
400 / 624 batches done
600 / 624 batches done
Epoch   1 | avg loss   0.0284 | train acc 93.12 | val acc 96.80
200 / 624 batches done
400 / 624 batches done
600 / 624 batches done
Epoch   2 | avg loss   0.0070 | train acc 98.46 | val acc 97.00
200 / 624 batches done
400 / 624 batches done
600 / 624 batches done
Epoch   3 | avg loss   0.0024 | train acc 99.46 | val acc 98.00
Final avg loss   0.0024 | final train acc 99.46 | best val acc 98.00
time: 20min 32s (started: 2021-04-15 13:35:28 +00:00)


## NVDM

Here we implement NVDM as a standalone topic modeler. NVDM is a neural VAE. The goal is to concatenate the output of NVDM's encoder, the posterior distribution $q(\mathbf{h} | \mathbf{B})$ with BERT's output (the embedding of the `[CLS]` token), and then fine-tune a joint loss between the two. $\mathbf{B}$ is the BOW representation of a document $\mathcal{D}$.

In [15]:
class NVDM(nn.Module):
  def __init__(self, vocab_size, num_topics, hidden_size=256, nonlinearity=nn.Tanh):
    super().__init__()
    self.num_topics = num_topics
    self.vocab_size = vocab_size

    # First MLP layer compresses from vocab_size to hidden_size
    mlp_layers = [nn.Linear(vocab_size, hidden_size), nonlinearity()]
    # Remaining layers operate in dimension hidden_size
    for _ in range(hidden_size - 1):
      mlp_layers.append(nn.Linear(hidden_size, hidden_size))
      mlp_layers.append(nonlinearity())

    self.mlp = nn.Sequential(*mlp_layers)

    # Create linear projections for Gaussian params (mean & sigma)
    self.mean = nn.Linear(vocab_size, num_topics)
    # These 0 initializers may be necessary... let's wait on performance to decide
    # self.mean.bias.data.zero_()
    # self.mean.weight.data.fill_(0.)
    self.log_sigma = nn.Linear(vocab_size, num_topics)
    # self.sigma.bias.data.zero_()
    # self.sigma.weight.data.fill_(0.)

    self.projection = nn.Linear(num_topics, vocab_size)
    self.log_softmax = nn.LogSoftmax(-1)



  def forward(self, input_bows, mask, labels):
    # Run BOW through MLP
    pi = mlp(input_bows)

    # Use this to get mean, log_sig for Gaussian
    mean = self.mean(pi)
    log_sigma = self.log_sigma(pi)

    # Calculate KLD
    kld = -0.5 * torch.sum(1 - torch.square(mean) + (2 * log_sigma - torch.exp(2 * log_sigma)), 1)
    kld = mask * kld  # mask paddings

    # Use Gaussian reparam. trick to sample from distribution defined by mu, sig
    # This provides a sample h_tm from posterior q(h_tm | V) (tm meaning topic model)
    epsilons = torch.normal(0, 1, size=(self.vocab_size, self.num_topics))
    sample = (torch.exp(log_sigma) * epsilons) + mean

    # Softmax to get p(v_i | h_tm), AKA probabilities of words given hidden state
    logits = self.log_softmax(self.projection(sample)) 

    # Lowerbound on NVDM true loss, used for optimization
    rec_loss = -1 * torch.sum(logits * input_bows, 1)
    loss_nvdm_lb = torch.mean(rec_loss + kld)

    return sample, loss_nvdm_lb


In [20]:
# def train(sess, model, 
#           train_url, 
#           test_url, 
#           batch_size, 
#           training_epochs=1000, 
#           alternate_epochs=10):
#   """train nvdm model."""
#   train_set, train_count = utils.data_set(train_url)
#   test_set, test_count = utils.data_set(test_url)
#   # hold-out development dataset
#   dev_set = test_set[:50]
#   dev_count = test_count[:50]

#   dev_batches = utils.create_batches(len(dev_set), batch_size, shuffle=False)
#   test_batches = utils.create_batches(len(test_set), batch_size, shuffle=False)
  
#   for epoch in range(training_epochs):
#     train_batches = utils.create_batches(len(train_set), batch_size, shuffle=True)
#     #-------------------------------
#     # train
#     for switch in xrange(0, 2):
#       if switch == 0:
#         optim = model.optim_dec
#         print_mode = 'updating decoder'
#       else:
#         optim = model.optim_enc
#         print_mode = 'updating encoder'
#       for i in xrange(alternate_epochs):
#         loss_sum = 0.0
#         ppx_sum = 0.0
#         kld_sum = 0.0
#         word_count = 0
#         doc_count = 0
#         for idx_batch in train_batches:
#           data_batch, count_batch, mask = utils.fetch_data(
#           train_set, train_count, idx_batch, FLAGS.vocab_size)
#           input_feed = {model.x.name: data_batch, model.mask.name: mask}
#           _, (loss, kld) = sess.run((optim, 
#                                     [model.objective, model.kld]),
#                                     input_feed)
#           loss_sum += np.sum(loss)
#           kld_sum += np.sum(kld) / np.sum(mask) 
#           word_count += np.sum(count_batch)
#           # to avoid nan error
#           count_batch = np.add(count_batch, 1e-12)
#           # per document loss
#           ppx_sum += np.sum(np.divide(loss, count_batch)) 
#           doc_count += np.sum(mask)
#         print_ppx = np.exp(loss_sum / word_count)
#         print_ppx_perdoc = np.exp(ppx_sum / doc_count)
#         print_kld = kld_sum/len(train_batches)
#         print('| Epoch train: {:d} |'.format(epoch+1), 
#                print_mode, '{:d}'.format(i),
#                '| Corpus ppx: {:.5f}'.format(print_ppx),  # perplexity for all docs
#                '| Per doc ppx: {:.5f}'.format(print_ppx_perdoc),  # perplexity for per doc
#                '| KLD: {:.5}'.format(print_kld))

# TODO: write suitable train code & debug NVDM implementationn
# No LR scheduling, 2 Adam optimizers (torch.optim.Adam) — one for encoding
# params, and another for decoding params. Alternate which one is being updated

def train_nvdm(model, batch_size=32, lr=0.00005, num_epochs=3, verbose=True, device='cuda'):
  model = model.to(device)  # Move the model to device.  
  dataloader_train = DataLoader(r8_train, batch_size=batch_size, shuffle=True, num_workers=2, collate_fn=collate_fn) 
  num_train_steps = len(r8_train) // batch_size * num_epochs
  optimizer, scheduler = configure_optimization(model, num_train_steps, num_warmup_steps, lr)

  loss_avg = float('inf')
  acc_train = 0.
  best_acc_val = 0.
  for epoch in range(num_epochs):
    model.train()  # This turns on the training mode (e.g., enable dropout).
    loss_total = 0.
    num_correct_train = 0
    for batch_ind, (input_ids, attention_mask, labels) in enumerate(dataloader_train):
      if (batch_ind + 1) % 200 == 0: 
        print(batch_ind + 1, '/', len(dataloader_train), 'batches done')
      input_ids = input_ids.to(device).long()
      attention_mask = attention_mask.to(device)
      labels = labels.to(device)      
      logits, loss_batch_total = model(input_ids, attention_mask, labels) 
      preds = torch.max(logits, 1)[1]  # get label from max score per example
      num_correct_train += (preds == torch.max(labels, 1)[1]).sum()
      loss_total += loss_batch_total.item()            
      
      loss_batch_avg = loss_batch_total / input_ids.size(0)  
      loss_batch_avg.backward()  

      optimizer.step()  # optimizer updates model weights based on stored gradients
      scheduler.step()  # Update lr. 
      optimizer.zero_grad()  # Reset gradient slots to zero

    # Useful training information
    loss_avg = loss_total / len(dataloader_train.dataset)
    acc_train = num_correct_train / len(dataloader_train.dataset) * 100.

    # Check validation performance at the end of every epoch. 
    acc_val = get_acc_val(model, device)

    if verbose:
      print('Epoch {:3d} | avg loss {:8.4f} | train acc {:2.2f} | val acc {:2.2f}'.format(epoch + 1, loss_avg, acc_train, acc_val))

    if acc_val > best_acc_val: 
      best_acc_val = acc_val
  
  if verbose: 
    print('Final avg loss {:8.4f} | final train acc {:2.2f} | best val acc {:2.2f}'.format(loss_avg, acc_train, best_acc_val))
    
  return best_acc_val